In [2]:
import pandas as pd
from neo4j import GraphDatabase
from openai import OpenAI
from dotenv import load_dotenv
import os
from utils import *
import json

# Load environment variables from .env file
load_dotenv()

True

In [5]:
# Get the URI and authentication credentials from environment variables
URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))

# initalize the driver
driver = GraphDatabase.driver(URI, auth=AUTH)


In [ ]:
import faiss
import numpy as np

# Load FAISS index
faiss_index = faiss.read_index("graph_embeddings.index")

# Node names (stored separately during creation)
with open("node_names.txt", "r") as f:
    node_names = [line.strip() for line in f.readlines()]

# Extract embedding vectors
embeddings = faiss_index.reconstruct_n(0, len(node_names))

# Convert to a list of dictionaries for Neo4j import
embedding_data = [
    {"name": node_names[i], "embedding": embeddings[i].tolist()}
    for i in range(len(node_names))
]

# Example output
print(embedding_data[:2])
# [{'name': 'city', 'embedding': [0.123, 0.456, ...]}, ...]

In [ ]:
def add_embeddings_to_neo4j(embedding_data, driver):
    """
    Add embedding vectors to nodes in Neo4j Aura.
    Args:
        embedding_data: List of dictionaries with node names and embeddings.
        driver: Neo4j driver instance.
    """
    query = """
    MATCH (n {name: $name})
    SET n.embedding = $embedding
    """
    with driver.session() as session:
        for data in embedding_data:
            session.run(query, name=data["name"], embedding=data["embedding"])
    print("Embeddings added to Neo4j.")


In [ ]:
# Add embeddings
add_embeddings_to_neo4j(embedding_data, driver)

Embeddings added to Neo4j.


In [8]:
# create vector index in neo4j
def create_vector_index(driver):
    """
    Create a vector index for node embeddings in Neo4j.
    Args:
        driver: The Neo4j driver instance.
    """
    cypher_query = """
    CREATE INDEX node_embedding_index
    FOR (n:Node) ON n.embedding
    OPTIONS {
      "indexConfig": {
        "algorithm": "HNSW",
        "similarityFunction": "COSINE"
      }
    }
    """
    with driver.session() as session:
        session.run(cypher_query)
        print("Vector index created successfully.")

# Create the index
create_vector_index(driver)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '"indexConfig"': expected an identifier or '}' (line 5, column 7 (offset: 91))
"      "indexConfig": {"
       ^}